In [10]:
import time
import csv
from bs4 import BeautifulSoup
from seleniumbase import SB
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def bypass_cloudflare_and_goto_timetable(url, target_keyword):
    with SB(uc=True, headless=False) as sb:
        driver = sb.driver
        driver.set_window_size(1920, 1080)

        print(f"🌐 Mở: {url}")
        driver.get(url)

        print("⏳ Chờ vượt Cloudflare...")
        time.sleep(10)

        print(f"🕵️ Đang theo dõi URL để tìm '{target_keyword}'...")
        matched = False
        while True:
            try:
                current_url = driver.current_url
                if target_keyword in current_url and not matched:
                    print(f"🎯 Tìm thấy URL chứa '{target_keyword}': {current_url}")
                    matched = True

                    with open("matched_page.html", "w", encoding="utf-8") as f:
                        f.write(driver.page_source)
                    print("💾 HTML đã được lưu vào matched_page.html")

                    # 🔽 Truy cập trang thời khóa biểu
                    timetable_url = "https://fap.fpt.edu.vn/Course/Courses.aspx"
                    print(f"🚀 Đang chuyển tới: {timetable_url}")
                    driver.get(timetable_url)
                    time.sleep(5)

                    print("👉 Trước khi đợi phần tử 'Artificial Intelligence'")
                    ai_element = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.LINK_TEXT, "Artificial Intelligence"))
                    )
                    print("✅ Đã tìm thấy phần tử 'Artificial Intelligence'")
                    ai_element.click()

                    print("👉 Đang đợi bảng thời khóa biểu xuất hiện...")
                    time.sleep(10)  # Đợi bảng xuất hiện

                    # 👉 Trích HTML hiện tại
                    html = driver.page_source
                    with open("debug_ai_page.html", "w", encoding="utf-8") as f:
                        f.write(html)

                    soup = BeautifulSoup(html, "html.parser")

                    # 🧠 Tìm bảng có class table-bordered
                    table = soup.find("table", class_="table-bordered")
                    if table:
                        print("✅ Tìm thấy bảng thời khóa biểu")

                        headers = [th.get_text(strip=True) for th in table.find_all("th")]
                        print("📋 Tiêu đề cột:", headers)

                        rows = table.find_all("tr")
                        print(f"📌 Có {len(rows)} hàng")

                        # 🔽 Ghi ra CSV
                        with open("schedule_ai.csv", "w", newline="", encoding="utf-8") as csvfile:
                            writer = csv.writer(csvfile)
                            writer.writerow(headers)
                            for row in rows[1:]:
                                cols = [td.get_text(strip=True) for td in row.find_all("td")]
                                if cols:
                                    writer.writerow(cols)
                        print("💾 Đã lưu file: schedule_ai.csv")
                    else:
                        print("❌ Không tìm thấy bảng thời khóa biểu")
                driver.title
                time.sleep(1)
            except:
                print("🔴 Trình duyệt đã đóng hoặc lỗi.")
                break

# 🟢 Gọi hàm
bypass_cloudflare_and_goto_timetable(
    "https://fap.fpt.edu.vn", target_keyword="https://fap.fpt.edu.vn/Student.aspx"
)


🌐 Mở: https://fap.fpt.edu.vn
⏳ Chờ vượt Cloudflare...
🕵️ Đang theo dõi URL để tìm 'https://fap.fpt.edu.vn/Student.aspx'...
🎯 Tìm thấy URL chứa 'https://fap.fpt.edu.vn/Student.aspx': https://fap.fpt.edu.vn/Student.aspx
💾 HTML đã được lưu vào matched_page.html
🚀 Đang chuyển tới: https://fap.fpt.edu.vn/Course/Courses.aspx
👉 Trước khi đợi phần tử 'Artificial Intelligence'
✅ Đã tìm thấy phần tử 'Artificial Intelligence'
👉 Đang đợi bảng thời khóa biểu xuất hiện...
✅ Tìm thấy bảng thời khóa biểu
📋 Tiêu đề cột: ['Subject', 'Student', 'Plan']
📌 Có 18 hàng
💾 Đã lưu file: schedule_ai.csv
🔴 Trình duyệt đã đóng hoặc lỗi.
